### OHLCV Data Cleaning Pipeline

This notebook cleans the consolidated OHLCV data by ensuring data integrity and temporal alignment across all tickers.

**Workflow:**

1.  **Prerequisite:** A consolidated OHLCV Parquet file (e.g., `df_OHLCV_stocks_etfs.parquet`) must exist.
2.  **Load Data:** The raw, consolidated OHLCV data is loaded from the Parquet file.
3.  **Clean & Filter:** The data undergoes a two-step cleaning process:
    *   **Date Alignment:** All tickers are aligned to a common date index based on a stable reference symbol (e.g., 'VOO'). Tickers with mismatched dates are removed.
    *   **Completeness Check:** Any remaining tickers with `NaN` values or an incomplete date range are removed.
4.  **Save Data:** The final, clean DataFrame is saved to a new Parquet file.
5.  **Summarize:** A final report is generated detailing the number of tickers at each stage and listing those that were filtered out.

### Setup and Configuration

**This is the only cell you need to modify.** Adjust the filenames and reference symbol as needed for your project.

In [4]:
import sys
from pathlib import Path
import pandas as pd

# --- Project Path Setup ---
# Autodetect the project's root directory.
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR
DATA_DIR = ROOT_DIR / 'data'
SRC_DIR = ROOT_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# Import custom utility functions now that the path is set
import utils

# --- File and Cleaning Configuration ---
SOURCE_FILENAME = 'df_OHLCV_stocks_etfs.parquet'
DEST_FILENAME = 'df_OHLCV_clean_stocks_etfs.parquet'
REFERENCE_SYMBOL = 'VOO'  # Use this ticker's date index as the gold standard

# --- Construct Full Paths ---
SOURCE_PATH = DATA_DIR / SOURCE_FILENAME
DEST_PATH = DATA_DIR / DEST_FILENAME

# --- Notebook Setup ---
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
%load_ext autoreload
%autoreload 2

# --- Verification ---
print(f"Project Root Directory: {ROOT_DIR}")
print(f"Source file: {SOURCE_PATH}")
print(f"Destination file: {DEST_PATH}")
print(f"Reference symbol for date alignment: '{REFERENCE_SYMBOL}'")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Project Root Directory: c:\Users\ping\Files_win10\python\py311\stocks
Source file: c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet
Destination file: c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_clean_stocks_etfs.parquet
Reference symbol for date alignment: 'VOO'


### Step 1: Load Raw OHLCV Data

Load the consolidated OHLCV data and perform an initial validation to ensure the reference symbol exists.

In [5]:
print(f"--- Step 1: Loading raw data from {SOURCE_PATH.name} ---")

try:
    df_raw = pd.read_parquet(SOURCE_PATH, engine='pyarrow')
    raw_ticker_count = len(df_raw.index.get_level_values('Ticker').unique())
    print(f"Successfully loaded data with {raw_ticker_count} unique tickers.")
    
    # --- CRITICAL VALIDATION ---
    # Check if the reference symbol exists before proceeding.
    if REFERENCE_SYMBOL not in df_raw.index.get_level_values('Ticker'):
        raise ValueError(f"Reference symbol '{REFERENCE_SYMBOL}' not found in the raw data. Halting execution.")
    else:
        print(f"Validation successful: Reference symbol '{REFERENCE_SYMBOL}' is present.")

    df_raw.info()
    display(df_raw.head(3))
    
except FileNotFoundError:
    print(f"ERROR: Source file not found at {SOURCE_PATH}")
    df_raw = None
except Exception as e:
    print(f"An error occurred: {e}")
    df_raw = None

--- Step 1: Loading raw data from df_OHLCV_stocks_etfs.parquet ---
Successfully loaded data with 1626 unique tickers.
Validation successful: Reference symbol 'VOO' is present.
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 552884 entries, ('A', Timestamp('2025-06-13 00:00:00')) to ('ZWS', Timestamp('2024-02-01 00:00:00'))
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Open   552884 non-null  float64
 1   Adj High   552884 non-null  float64
 2   Adj Low    552884 non-null  float64
 3   Adj Close  552884 non-null  float64
 4   Volume     552884 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 23.3+ MB


Adj Open  Adj High  Adj Low  Adj Close   Volume
Ticker Date                                                       
A      2025-06-13    116.68    118.90   116.45     117.08  1652000
       2025-06-12    118.98    120.05   118.53     118.91  1361500
       2025-06-11    120.76    122.83   119.67     119.72  2752400

### Step 2: Clean and Filter Data

Apply the two-stage cleaning process using the custom utility functions. After each utility call, we restore the index names.

In [6]:
if df_raw is not None:
    print("\n--- Step 2: Starting data cleaning and filtering ---")
    
    # --- Part A: Align dates to the reference symbol ---
    print("\nPart A: Filtering dates to align with the reference symbol...")
    df_filtered, filtered_out_symbols = utils.filter_df_dates_to_reference_symbol(
        df=df_raw,
        reference_symbol=REFERENCE_SYMBOL
    )
    
    # FIX: Restore index names, as they can be lost by the utility function.
    df_filtered.index.names = ['Ticker', 'Date']
    
    filter_ticker_count = len(df_filtered.index.get_level_values('Ticker').unique())
    print(f"Removed {len(filtered_out_symbols)} tickers with non-matching date ranges.")
    print(f"Tickers remaining after date alignment: {filter_ticker_count}")
    
    # --- Part B: Remove symbols with missing values or incomplete date ranges ---
    print("\nPart B: Filtering symbols with missing values (NaNs) or incomplete data...")
    df_clean, missing_values_symbols = utils.filter_symbols_with_missing_values(
        df=df_filtered
    )

    # FIX: Restore index names again for the final clean DataFrame.
    df_clean.index.names = ['Ticker', 'Date']

    clean_ticker_count = len(df_clean.index.get_level_values('Ticker').unique())
    print(f"Removed {len(missing_values_symbols)} tickers with missing data points.")
    print(f"Tickers remaining after final cleaning: {clean_ticker_count}")

else:
    print("Skipping cleaning step because raw data failed to load.")
    df_clean = None


--- Step 2: Starting data cleaning and filtering ---

Part A: Filtering dates to align with the reference symbol...
Using 'Ticker' as the symbol identifier.
Original number of Tickers: 1626
Number of Tickers after filtering: 1556
Number of Tickers filtered out: 70

First 10 Tickers that were filtered out:
['JGLO', 'QFIN', 'CROX', 'SFD', 'TFX', 'KRMN', 'CRWV', 'ETHA', 'VG', 'ULS']

Example of dates for first filtered out Ticker:

Dates for JGLO:
DatetimeIndex(['2025-06-13', '2025-06-12', '2025-06-11', '2025-06-10', '2025-06-09', '2025-06-06', '2025-06-05', '2025-06-04', '2025-06-03', '2025-06-02',
               ...
               '2024-03-26', '2024-03-25', '2024-03-22', '2024-03-21', '2024-03-20', '2024-03-19', '2024-03-18', '2024-03-15', '2024-03-14', '2024-03-13'], dtype='datetime64[ns]', name='Date', length=315, freq=None)

Filtered DataFrame info:
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 533708 entries, ('A', Timestamp('2025-06-13 00:00:00')) to ('ZWS', Timestamp('2024-0

### Step 3: Save Cleaned Data

Save the fully cleaned DataFrame to a new Parquet file.

In [8]:
if df_clean is not None and not df_clean.empty:
    print(f"\n--- Step 3: Saving cleaned data ---")
    try:
        # Ensure the destination directory exists
        DEST_PATH.parent.mkdir(parents=True, exist_ok=True)
        
        df_clean.to_parquet(DEST_PATH, engine='pyarrow', compression='zstd')
        print(f"Successfully saved cleaned data to: {DEST_PATH}")
        
    except Exception as e:
        print(f"Error: Failed to save Parquet file. Details: {e}")
else:
    print("\nSkipping save step because the cleaned DataFrame is empty or does not exist.")


--- Step 3: Saving cleaned data ---
Successfully saved cleaned data to: c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_clean_stocks_etfs.parquet


### Step 4: Final Summary

Provide a complete summary of the cleaning process, showing the number of tickers at each stage and listing those that were removed.


In [9]:
if 'df_raw' in locals() and df_raw is not None:
    print("\n--- Step 4: Cleaning Process Summary ---")
    
    print(f"Initial ticker count:           {raw_ticker_count}")
    print(f"Tickers after date alignment:   {filter_ticker_count}")
    print(f"Final clean ticker count:       {clean_ticker_count}")
    print("-" * 40)
    print(f"Total tickers removed:          {raw_ticker_count - clean_ticker_count}")
    
    print(f"\n{len(filtered_out_symbols)} symbols removed due to non-matching date index:")
    print(sorted(filtered_out_symbols))
    
    print(f"\n{len(missing_values_symbols)} symbols removed due to missing values or incomplete data:")
    print(sorted(missing_values_symbols))
else:
    print("\nSummary could not be generated as raw data was not loaded.")



--- Step 4: Cleaning Process Summary ---
Initial ticker count:           1626
Tickers after date alignment:   1556
Final clean ticker count:       1556
----------------------------------------
Total tickers removed:          70

70 symbols removed due to non-matching date index:
['ACIW', 'ADMA', 'ALAB', 'BMA', 'BTC', 'BULL', 'CADE', 'CRCL', 'CROX', 'CRWV', 'CTRE', 'CZR', 'DAR', 'DFS', 'EMLP', 'ETHA', 'ETOR', 'EWU', 'FCFS', 'FCN', 'FNGA', 'GEV', 'GLOB', 'GOLD', 'GPI', 'HOMB', 'JGLO', 'KRMN', 'LB', 'LINE', 'LNC', 'LNTH', 'LOAR', 'LTM', 'MMSI', 'MNSO', 'MSTY', 'NBIS', 'ONTO', 'OS', 'PCG', 'PTLC', 'QFIN', 'QTWO', 'RBRK', 'RDDT', 'SAIC', 'SAIL', 'SARO', 'SATS', 'SFD', 'SMBS', 'SNDK', 'SOLV', 'SPHD', 'SPSC', 'SQQQ', 'SW', 'TEM', 'TFI', 'TFX', 'TLX', 'TTAN', 'UCON', 'ULS', 'VG', 'VIK', 'VNQI', 'WAY', 'ZK']

0 symbols removed due to missing values or incomplete data:
[]
